In [6]:
%pylab inline
import time
from tqdm import tqdm

Populating the interactive namespace from numpy and matplotlib


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
def get_box(key):
    for r_idx, r in enumerate(ROW):
        for c_idx, c in enumerate(COL):
            if (key[0] == r) and (key[-1] == c):    
                return boxes[r_idx][c_idx]

def check_cols(n, col, board):
    for key in board.keys():
        if key[-1] == col:
            if board[key] == n:
                return False
    return True

def check_rows(n, row, board):
    for key in board.keys():
        if key[0] == row:
            if board[key] == n:
                return False
    return True

def check_box(n, box):
    global box_to_vals
    if n in box_to_vals[box]:
        return False
    return True
    

In [3]:
def is_legal(n, coord, board):
    
    return (check_cols(n, coord[-1], board) and check_rows(n, coord[0], board) \
            and check_box(n, get_box(coord)) and (n>0) and (n<10))\
            and (board[coord]==0)

In [4]:
def is_complete(board):
    return (not 0 in board.values())

def all_legal_vals(coord, board):
    legal_vals = []
    for n in range(1,10):
        if is_legal(n, coord, board):
            legal_vals.append(n)
    return legal_vals

def get_mrv(open_coords, board):
    """
    Returns the unfilled box coordinate (coord) with minimum remaining [legal] values (MRV) 
    and the corresponding values (available domain)
    """
    #Could this be improved?
    possible_moves = [(all_legal_vals(coord, board), coord) for coord in open_coords]
    n_moves_per_box = [len(moves[0]) for moves in possible_moves]
    idx = np.argmin(n_moves_per_box)
    return open_coords[idx], possible_moves[idx][0]

def get_mrv_improved():
    #Sort list of available moves from global moveset dict.
    #return minimum.
    to_return = sorted([(len(v), v, k) for k,v in moveset.items() if v])
    if to_return:
        return to_return[0]
    else:
        return


def solve_mrv(board):
    
    b = board.copy()
    
    if is_complete(b):
        return b
    
    '''
    get coordinates of available spaces to fill
    '''
    
    _, vals, coord = get_mrv_improved()
    print(vals, coord)
    
    for value in vals:
        
        b[coord] = value
        
        old_moveset_values = moveset[coord]
        
        moveset[coord] = set()
        
        result = solve_mrv(b)
        if result:
            return result
        else:
            b[coord] = 0
            moveset[coord] = old_moveset_values
    return False

In [8]:
ROW = "ABCDEFGHI"
COL = "123456789"

def to_np(inp):
    bd = np.array(list(inp.values())).reshape((9,9))
    return bd

def to_dict(line):
    global ROW
    global COL
    return {ROW[r] + COL[c]: int(line[9*r+c]) for r in range(9) for c in range(9)}

sudokus = []
with open('starter/sudokus_start.txt', 'r+') as f:
    sudokus = [to_dict(i.strip()) for i in f.readlines()]
    
    

boxes = [[(i//3,j//3) for i in range(9)] for j in range(9)]
box_to_vals = dict()
solution_data = dict()

In [10]:
board = sudokus[0]
to_np(board)

array([[0, 0, 3, 0, 2, 0, 6, 0, 0],
       [9, 0, 0, 3, 0, 5, 0, 0, 1],
       [0, 0, 1, 8, 0, 6, 4, 0, 0],
       [0, 0, 8, 1, 0, 2, 9, 0, 0],
       [7, 0, 0, 0, 0, 0, 0, 0, 8],
       [0, 0, 6, 7, 0, 8, 2, 0, 0],
       [0, 0, 2, 6, 0, 9, 5, 0, 0],
       [8, 0, 0, 2, 0, 3, 0, 0, 9],
       [0, 0, 5, 0, 1, 0, 3, 0, 0]])

In [ ]:
all_legal_vals(coord, board)

In [11]:
for key in board.keys():
    val = board[key]
    box = get_box(key)
    if box_to_vals.get(box):
        box_to_vals[box].append(val)
    else:
        box_to_vals[box] = [val]


moveset = {coord:set(all_legal_vals(coord, board)) for coord, val in board.items()}
print(to_np(board))
#solve_mrv(board)
print(moveset)


[[0 0 3 0 2 0 6 0 0]
 [9 0 0 3 0 5 0 0 1]
 [0 0 1 8 0 6 4 0 0]
 [0 0 8 1 0 2 9 0 0]
 [7 0 0 0 0 0 0 0 8]
 [0 0 6 7 0 8 2 0 0]
 [0 0 2 6 0 9 5 0 0]
 [8 0 0 2 0 3 0 0 9]
 [0 0 5 0 1 0 3 0 0]]
{'A1': {4, 5}, 'A2': {8, 4, 5, 7}, 'A3': set(), 'A4': {9, 4}, 'A5': set(), 'A6': {1, 4, 7}, 'A7': set(), 'A8': {8, 9, 5, 7}, 'A9': {5, 7}, 'B1': set(), 'B2': {2, 4, 6, 7, 8}, 'B3': {4, 7}, 'B4': set(), 'B5': {4, 7}, 'B6': set(), 'B7': {8, 7}, 'B8': {8, 2, 7}, 'B9': set(), 'C1': {2, 5}, 'C2': {2, 5, 7}, 'C3': set(), 'C4': set(), 'C5': {9, 7}, 'C6': set(), 'C7': set(), 'C8': {2, 3, 5, 7, 9}, 'C9': {2, 3, 5, 7}, 'D1': {3, 4, 5}, 'D2': {3, 4, 5}, 'D3': set(), 'D4': set(), 'D5': {3, 4, 5, 6}, 'D6': set(), 'D7': set(), 'D8': {3, 4, 5, 6, 7}, 'D9': {3, 4, 5, 6, 7}, 'E1': set(), 'E2': {1, 2, 3, 4, 5, 9}, 'E3': {9, 4}, 'E4': {9, 4, 5}, 'E5': {3, 4, 5, 6, 9}, 'E6': {4}, 'E7': {1}, 'E8': {1, 3, 4, 5, 6}, 'E9': set(), 'F1': {1, 3, 4, 5}, 'F2': {1, 3, 4, 5, 9}, 'F3': set(), 'F4': set(), 'F5': {9, 3, 4, 5}, 'F6':

In [ ]:

run = False

if run:
    for idx, board in tqdm(enumerate(sudokus[:2])):
        #Reset pertinent globals:
        box_to_vals = dict()

        #board = sudokus[16]

        #Prep grid reference:
        for key in board.keys():
            val = board[key]
            box = get_box(key)
            if box_to_vals.get(box):
                box_to_vals[box].append(val)
            else:
                box_to_vals[box] = [val]

        start = time.time()
        solution = solve_mrv(board)
        end = time.time()

        time_to_solve = end-start

        solution_data[idx] = {'solution':solution, 'time':time_to_solve}

        if solution:
            print(to_np(solution))
        else:
            print('No solution found.')

        print(f'{(time_to_solve)} sec')


In [49]:
'''
Experiments in vectorization (np)
'''


'''def get_box(key):
    #Implement this.
    
    for r_idx, r in enumerate(ROW):
        for c_idx, c in enumerate(COL):
            if (key[0] == r) and (key[-1] == c):    
                return boxes[r_idx][c_idx]
            '''
            
def check_cols_np(n, col, board):
    return (not np.any(board[:,col]==n))